# XGB Classifier with Wrapper-Based FS

Steven Sison | March 9, 2024

## Description

This document will be used to train a model using the reduced feature set obtain by using the wrapper-based method, forward feature selection. The model will be evaluated in terms of the usual metrics (accuracy, precision, F1-score, recall) as well as the training time. The model will also be stored for future evaluation purposes.

## Training the Model

### Preliminaries

#### 1. Loading the Dataset

In [1]:
import pandas as pd                     # For data transformation
import numpy as numpy                   # For scientific calculations
import seaborn as sns                   # For data visualizations
import matplotlib.pyplot as plt         # For plotting
import plotly.graph_objects as go       # For plotting
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, ConfusionMatrixDisplay
from xgboost import XGBClassifier, DMatrix, train
from sklearn.pipeline import Pipeline
import time
from datetime import datetime
import joblib
import os

dataset = pd.read_csv("../../../02_feature-engineering/final-datasets/binary_new_Bacud_unbalanced_lexical.csv")      # Loading the dataset

dataset.head()



,url_type,url_length,url_ip_in_domain,url_domain_entropy,url_is_digits_in_domain,url_query_length,url_number_of_parameters,url_number_of_digits,url_string_entropy,url_is_https,...,has_swf_in_string,has_cgi_in_string,has_php_in_string,has_abuse_in_string,has_admin_in_string,has_bin_in_string,has_personal_in_string,has_update_in_string,has_verification_in_string,url_scheme
0,1,16,0,3.169925,0,0,0,0,3.375000,0,...,0,0,0,0,0,0,0,0,0,0
1,0,35,0,2.807355,1,0,0,1,4.079143,0,...,0,0,0,0,0,0,0,0,0,0
2,0,31,0,2.921928,0,0,0,1,3.708093,0,...,0,0,0,0,0,0,0,0,0,0
3,1,88,0,2.896292,0,49,4,7,4.660343,0,...,0,0,1,0,0,0,0,0,0,27
4,1,235,0,3.405822,0,194,3,22,4.980518,0,...,0,0,1,0,0,0,0,0,0,27


In [2]:
dataset.head()

,url_type,url_length,url_ip_in_domain,url_domain_entropy,url_is_digits_in_domain,url_query_length,url_number_of_parameters,url_number_of_digits,url_string_entropy,url_is_https,...,has_swf_in_string,has_cgi_in_string,has_php_in_string,has_abuse_in_string,has_admin_in_string,has_bin_in_string,has_personal_in_string,has_update_in_string,has_verification_in_string,url_scheme
0,1,16,0,3.169925,0,0,0,0,3.375000,0,...,0,0,0,0,0,0,0,0,0,0
1,0,35,0,2.807355,1,0,0,1,4.079143,0,...,0,0,0,0,0,0,0,0,0,0
2,0,31,0,2.921928,0,0,0,1,3.708093,0,...,0,0,0,0,0,0,0,0,0,0
3,1,88,0,2.896292,0,49,4,7,4.660343,0,...,0,0,1,0,0,0,0,0,0,27
4,1,235,0,3.405822,0,194,3,22,4.980518,0,...,0,0,1,0,0,0,0,0,0,27


In [3]:
x_train, x_test, y_train, y_test = train_test_split(dataset.drop(columns=['url_type']), dataset['url_type'], test_size = 0.2, random_state=42)

#### 2. Preprocessing (Balancing)

In [4]:
dataset['url_type'].value_counts()

url_type
0    724778
1    380244
Name: count, dtype: int64

#### 3. Removing Unnecessary Features

In [5]:
important_features_wrapper_33 = ['url_length',
 'url_domain_entropy',
 'url_is_digits_in_domain',
 'url_number_of_parameters',
 'url_number_of_digits',
 'url_string_entropy',
 'url_path_length',
 'url_host_length',
 'get_tld',
 'url_domain_len',
 'url_num_subdomain',
 'url_number_of_fragments',
 'url_is_encoded',
 'url_number_of_letters',
 'url_num_periods',
 'url_num_of_hyphens',
 'url_num_underscore',
 'url_num_forward_slash',
 'url_num_semicolon',
 'url_num_mod_sign',
 'has_login_in_string',
 'has_signin_in_string',
 'has_logon_in_string',
 'has_loginasp_in_string',
 'has_exe_in_string',
 'has_viewerphp_in_string',
 'has_getImageasp_in_string',
 'has_paypal_in_string',
 'has_dbsysphp_in_string',
 'has_shopping_in_string',
 'has_php_in_string',
 'has_bin_in_string',
 'has_personal_in_string',
 'url_scheme'
 ]

important_features_wrapper_12 = ['url_domain_entropy', 
                              'url_number_of_parameters', 
                              'url_number_of_digits', 
                              'url_path_length', 
                              'url_host_length', 
                              'get_tld', 
                              'url_domain_len', 
                              'url_num_subdomain', 
                              'url_number_of_letters', 
                              'url_num_periods', 
                              'url_num_of_hyphens', 
                              'url_num_forward_slash', 
                              'url_num_semicolon', 
                              'has_login_in_string', 
                              'has_exe_in_string', 
                              'has_php_in_string', 
                              'url_scheme']

X_test_12 = x_test[important_features_wrapper_12]
X_train_12 = x_train[important_features_wrapper_12]

X_test_33 = x_test[important_features_wrapper_33]
X_train_33 = x_train[important_features_wrapper_33]

X_test_33.head()

,url_length,url_domain_entropy,url_is_digits_in_domain,url_number_of_parameters,url_number_of_digits,url_string_entropy,url_path_length,url_host_length,get_tld,url_domain_len,...,has_exe_in_string,has_viewerphp_in_string,has_getImageasp_in_string,has_paypal_in_string,has_dbsysphp_in_string,has_shopping_in_string,has_php_in_string,has_bin_in_string,has_personal_in_string,url_scheme
165686,58,1.921928,0,1,14,4.659537,32,0,0,5,...,0,0,0,0,0,0,0,0,0,0
712465,76,2.750000,0,0,10,4.626107,42,26,152,8,...,0,0,0,0,0,0,0,0,0,2
335773,133,2.664498,0,7,5,4.657590,10,21,332,14,...,0,0,0,0,0,0,1,0,0,27
533676,30,3.381580,0,0,0,3.989898,1,21,320,17,...,0,0,0,0,0,0,0,0,0,28
642230,44,2.699514,0,0,0,3.772185,44,0,202,14,...,0,0,0,0,0,0,0,0,0,0


#### Hyper-parameter Tuning

##### 12 Features (Purely Lexical)

In [6]:
import optuna
from sklearn.metrics import mean_squared_error # or any other metric
from sklearn.model_selection import train_test_split

# Define the objective function for Optuna
def objective_12(trial):
    # Define the search space for hyperparameters
    param = {
        'objective': 'binary:hinge',
        'eval_metric': 'error',
        'eta': trial.suggest_float('eta', 0.01, 0.3),
        'n_estimators': 100000, # Fix the boosting round and use early stopping
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0.0, 10.0),
        'min_child_weight': trial.suggest_float('min_child_weight', 0.1, 10.0),
        'lambda': trial.suggest_float('lambda', 0.1, 10.0),
        'alpha': trial.suggest_float('alpha', 0.0, 10.0),
    }
    
    # Split the data into further training and validation sets (three sets are preferable)
    train_data, valid_data, train_target, valid_target = train_test_split(X_train_12, y_train, test_size=0.2, random_state=42)
    
    # Convert the data into DMatrix format
    dtrain = DMatrix(train_data, label=train_target)
    dvalid = DMatrix(valid_data, label=valid_target)
    
    # Define the pruning callback for early stopping
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, 'validation-error')
    
    # Train the model with early stopping
    model = train(param, dtrain, num_boost_round=100000, evals=[(dvalid, 'validation')], early_stopping_rounds=100, callbacks=[pruning_callback])
    
    # Make predictions on the test set
    dtest = DMatrix(valid_data)
    y_pred = model.predict(dtest)
    
    # Calculate the root mean squared error
    error = mean_squared_error(valid_target, y_pred, squared=False)
    
    return error

# Create an Optuna study and optimize the objective function
study_12 = optuna.create_study(direction='minimize')
study_12.optimize(objective_12, n_trials=100) # Control the number of trials

# Print the best hyperparameters and the best RMSE
best_params_12 = study_12.best_params
best_error = study_12.best_value
print("Best Hyperparameters (12 Features): ", best_params_12)
print("Best Error (12 Features): ", best_error)

c:\Users\sison\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2024-04-08 11:30:34,032] A new study created in memory with name: no-name-facd2105-765c-4f2c-bf03-2c9eb903b8da


[0]	validation-error:0.65653


c:\Users\sison\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [11:30:35] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07f6e447eee219473-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


[1]	validation-error:0.65653
[2]	validation-error:0.65653
[3]	validation-error:0.65653
[4]	validation-error:0.65653
[5]	validation-error:0.65653
[6]	validation-error:0.65653
[7]	validation-error:0.65653
[8]	validation-error:0.65653
[9]	validation-error:0.65653
[10]	validation-error:0.65653
[11]	validation-error:0.65653
[12]	validation-error:0.65653
[13]	validation-error:0.65653
[14]	validation-error:0.65653
[15]	validation-error:0.65653
[16]	validation-error:0.65653
[17]	validation-error:0.65653
[18]	validation-error:0.65653
[19]	validation-error:0.65653
[20]	validation-error:0.65653
[21]	validation-error:0.63500
[22]	validation-error:0.60866
[23]	validation-error:0.51316
[24]	validation-error:0.51271
[25]	validation-error:0.42713
[26]	validation-error:0.34526
[27]	validation-error:0.31645
[28]	validation-error:0.24429
[29]	validation-error:0.20477
[30]	validation-error:0.18722
[31]	validation-error:0.15838
[32]	validation-error:0.14280
[33]	validation-error:0.12577
[34]	validation-err

[I 2024-04-08 11:30:40,086] Trial 0 finished with value: 0.3037904475188618 and parameters: {'eta': 0.029747240167371063, 'max_depth': 3, 'subsample': 0.7588276876169155, 'colsample_bytree': 0.5457520193022305, 'gamma': 7.663837697147277, 'min_child_weight': 0.6788038774925608, 'lambda': 4.12032003327361, 'alpha': 7.90961120421446}. Best is trial 0 with value: 0.3037904475188618.


...

[0]	validation-error:0.65653


c:\Users\sison\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [11:40:54] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07f6e447eee219473-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)
[I 2024-04-08 11:40:54,630] Trial 98 pruned. Trial was pruned at iteration 1.
c:\Users\sison\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [11:40:55] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07f6e447eee219473-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	validation-error:0.65653
[1]	validation-error:0.37273
[2]	validation-error:0.08789
[3]	validation-error:0.06621
[4]	validation-error:0.05920
[5]	validation-error:0.05569


[I 2024-04-08 11:40:55,612] Trial 99 pruned. Trial was pruned at iteration 6.


Best Hyperparameters (12 Features):  {'eta': 0.2601005317570466, 'max_depth': 10, 'subsample': 0.7087837126317342, 'colsample_bytree': 0.8843494832366752, 'gamma': 1.101644986306404, 'min_child_weight': 8.393799827019397, 'lambda': 6.136393348066621, 'alpha': 7.206222838062141}
Best Error (12 Features):  0.1852121015072687


##### 33 Features (Purely Lexical)

In [7]:
# Define the objective function for Optuna
def objective_33(trial):
    # Define the search space for hyperparameters
    param = {
        'objective': 'binary:hinge',
        'eval_metric': 'error',
        'eta': trial.suggest_float('eta', 0.01, 0.3),
        'n_estimators': 100000, # Fix the boosting round and use early stopping
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0.0, 10.0),
        'min_child_weight': trial.suggest_float('min_child_weight', 0.1, 10.0),
        'lambda': trial.suggest_float('lambda', 0.1, 10.0),
        'alpha': trial.suggest_float('alpha', 0.0, 10.0),
    }
    
    # Split the data into further training and validation sets (three sets are preferable)
    train_data, valid_data, train_target, valid_target = train_test_split(X_train_33, y_train, test_size=0.2, random_state=42)
    
    # Convert the data into DMatrix format
    dtrain = DMatrix(train_data, label=train_target)
    dvalid = DMatrix(valid_data, label=valid_target)
    
    # Define the pruning callback for early stopping
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, 'validation-error')
    
    # Train the model with early stopping
    model = train(param, dtrain, num_boost_round=100000, evals=[(dvalid, 'validation')], early_stopping_rounds=100, callbacks=[pruning_callback])
    
    # Make predictions on the test set
    dtest = DMatrix(valid_data)
    y_pred = model.predict(dtest)
    
    # Calculate the root mean squared error
    error = mean_squared_error(valid_target, y_pred, squared=False)
    
    return error

# Create an Optuna study and optimize the objective function
study_33 = optuna.create_study(direction='minimize')
study_33.optimize(objective_33, n_trials=100) # Control the number of trials

# Print the best hyperparameters and the best RMSE
best_params_33 = study_33.best_params
best_error_33 = study_33.best_value
print("Best Hyperparameters (33 Features): ", best_params_33)
print("Best Error (33 Features): ", best_error)

[I 2024-04-08 11:40:55,721] A new study created in memory with name: no-name-ce2d3e51-6bbb-44bf-9519-03635440248e
c:\Users\sison\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [11:40:57] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07f6e447eee219473-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	validation-error:0.65653
[1]	validation-error:0.63107
[2]	validation-error:0.16821
[3]	validation-error:0.10408
[4]	validation-error:0.09718
[5]	validation-error:0.07920
[6]	validation-error:0.07401
[7]	validation-error:0.06732
[8]	validation-error:0.06515
[9]	validation-error:0.06444
[10]	validation-error:0.06091
[11]	validation-error:0.06009
[12]	validation-error:0.05905
[13]	validation-error:0.05843
[14]	validation-error:0.05838
[15]	validation-error:0.05799
[16]	validation-error:0.05732
[17]	validation-error:0.05563
[18]	validation-error:0.05469
[19]	validation-error:0.05433
[20]	validation-error:0.05399
[21]	validation-error:0.05332
[22]	validation-error:0.05301
[23]	validation-error:0.05223
[24]	validation-error:0.05174
[25]	validation-error:0.05154
[26]	validation-error:0.05121
[27]	validation-error:0.05096
[28]	validation-error:0.05059
[29]	validation-error:0.05025
[30]	validation-error:0.05005
[31]	validation-error:0.04973
[32]	validation-error:0.04949
[33]	validation-erro

[I 2024-04-08 11:41:40,466] Trial 0 finished with value: 0.19114820139132388 and parameters: {'eta': 0.2735811033492584, 'max_depth': 6, 'subsample': 0.8081802467781581, 'colsample_bytree': 0.7250389940250708, 'gamma': 0.5228560211603905, 'min_child_weight': 0.35961839250876093, 'lambda': 2.3245992931269592, 'alpha': 6.526031100700437}. Best is trial 0 with value: 0.19114820139132388.
c:\Users\sison\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [11:41:41] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07f6e447eee219473-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	validation-error:0.65653
[1]	validation-error:0.47290
[2]	validation-error:0.10231
[3]	validation-error:0.07560
[4]	validation-error:0.06950
[5]	validation-error:0.06072
[6]	validation-error:0.05920
[7]	validation-error:0.05571
[8]	validation-error:0.05464
[9]	validation-error:0.05377
[10]	validation-error:0.05284
[11]	validation-error:0.05116
[12]	validation-error:0.05058
[13]	validation-error:0.04996
[14]	validation-error:0.04957
[15]	validation-error:0.04874
[16]	validation-error:0.04854
[17]	validation-error:0.04816
[18]	validation-error:0.04784
[19]	validation-error:0.04720
[20]	validation-error:0.04646
[21]	validation-error:0.04610
[22]	validation-error:0.04593
[23]	validation-error:0.04559
[24]	validation-error:0.04552
[25]	validation-error:0.04516
[26]	validation-error:0.04507
[27]	validation-error:0.04446
[28]	validation-error:0.04423
[29]	validation-error:0.04412
[30]	validation-error:0.04389
[31]	validation-error:0.04372
[32]	validation-error:0.04352
[33]	validation-erro

[I 2024-04-08 11:41:47,231] Trial 1 finished with value: 0.20441718353264243 and parameters: {'eta': 0.2796899928180234, 'max_depth': 9, 'subsample': 0.783124089974407, 'colsample_bytree': 0.5366246712143004, 'gamma': 8.671100289336016, 'min_child_weight': 6.733172037555324, 'lambda': 9.281812743134008, 'alpha': 7.6178545339328805}. Best is trial 0 with value: 0.19114820139132388.
c:\Users\sison\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [11:41:48] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07f6e447eee219473-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


...

[0]	validation-error:0.65653
[1]	validation-error:0.65653


[I 2024-04-08 12:10:48,792] Trial 98 pruned. Trial was pruned at iteration 1.
c:\Users\sison\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [12:10:49] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07f6e447eee219473-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	validation-error:0.65653
[1]	validation-error:0.65653


[I 2024-04-08 12:10:50,093] Trial 99 pruned. Trial was pruned at iteration 1.


Best Hyperparameters (33 Features):  {'eta': 0.29745962555816163, 'max_depth': 10, 'subsample': 0.8666733222849353, 'colsample_bytree': 0.5671549433917233, 'gamma': 0.7920764218697163, 'min_child_weight': 7.484658711592355, 'lambda': 0.9127396270051027, 'alpha': 5.749263792820111}
Best Error (33 Features):  0.1852121015072687


In [8]:
print(best_params_12)
print(best_params_33)

{'eta': 0.2601005317570466, 'max_depth': 10, 'subsample': 0.7087837126317342, 'colsample_bytree': 0.8843494832366752, 'gamma': 1.101644986306404, 'min_child_weight': 8.393799827019397, 'lambda': 6.136393348066621, 'alpha': 7.206222838062141}
{'eta': 0.29745962555816163, 'max_depth': 10, 'subsample': 0.8666733222849353, 'colsample_bytree': 0.5671549433917233, 'gamma': 0.7920764218697163, 'min_child_weight': 7.484658711592355, 'lambda': 0.9127396270051027, 'alpha': 5.749263792820111}


#### Model Training

In [9]:
from sklearn.model_selection import KFold
import optuna

# Initialize CV
cv = KFold(n_splits=10, shuffle=True, random_state=1)

# Set Hyperparameters
params_12 = { 'objective': 'binary:hinge',
              'eta': 0.2769198716475172, 
              'max_depth': 10, 
              'subsample': 0.82358642201105, 
              'colsample_bytree': 0.8286208231929323, 
              'gamma': 0.7766612702667438, 
              'min_child_weight': 8.319434489010376, 
              'lambda': 4.43143014244566, 
              'alpha': 4.399273966701367,
              'eval_metric': 'error'
              }

best_params_12['objective'] = 'binary:hinge'
best_params_12['eval_metric'] = 'error'

best_params_33['objective'] = 'binary:hinge'
best_params_33['eval_metric'] = 'error'

# Convert the data into DMatrix format
dtrain_12 = DMatrix(X_train_12, label=y_train)
dvalid_12 = DMatrix(X_test_12, label=y_test)

dtrain_33 = DMatrix(X_train_33, label=y_train)
dvalid_33 = DMatrix(X_test_33, label=y_test)

# Train the Model
xgb_classifier_12 = train(best_params_12, dtrain_12, num_boost_round=3000)
y_pred_12 = xgb_classifier_12.predict(dvalid_12)

print("Model with 12 Features Done.")

xgb_classifier_33 = train(best_params_33, dtrain_33, num_boost_round=3000)
y_pred_33 = xgb_classifier_33.predict(dvalid_33)

print("Model with 33 Features Done.")

Model with 12 Features Done.
Model with 33 Features Done.


#### Evaluation

In [10]:
# Classification Report
print(classification_report(y_test, y_pred_12))
print(classification_report(y_test, y_pred_33))

              precision    recall  f1-score   support

           0       0.97      0.98      0.97    144867
           1       0.96      0.95      0.95     76138

    accuracy                           0.97    221005
   macro avg       0.96      0.96      0.96    221005
weighted avg       0.97      0.97      0.97    221005

              precision    recall  f1-score   support

           0       0.97      0.98      0.98    144867
           1       0.96      0.95      0.96     76138

    accuracy                           0.97    221005
   macro avg       0.97      0.97      0.97    221005
weighted avg       0.97      0.97      0.97    221005



In [11]:
'''# Confusion Matrix for 12 Features
cm_up = confusion_matrix(y_test, y_pred_12, labels=xgb_classifier_12.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix = cm_up, display_labels = xgb_classifier_12.classes_)
disp.plot()
plt.show()'''

'# Confusion Matrix for 12 Features\ncm_up = confusion_matrix(y_test, y_pred_12, labels=xgb_classifier_12.classes_)\ndisp = ConfusionMatrixDisplay(confusion_matrix = cm_up, display_labels = xgb_classifier_12.classes_)\ndisp.plot()\nplt.show()'

In [12]:
'''# Cross Validation Score
scores = cross_val_score(XGBClassifier(**params_gbm),
                        X_train, y_train, scoring='accuracy', cv=cv).mean()

print(scores)'''

"# Cross Validation Score\nscores = cross_val_score(XGBClassifier(**params_gbm),\n                        X_train, y_train, scoring='accuracy', cv=cv).mean()\n\nprint(scores)"

In [13]:
# Dumping the model
joblib.dump(xgb_classifier_12, 'xgb_ffs_12.sav')
joblib.dump(xgb_classifier_33, 'xgb_ffs_33.sav')

['xgb_ffs_33.sav']

In [16]:
import lexical_generator_12
import lexical_generator_33
import time

def xgb_predict_maliciousness_12(url):

    numerical_values = lexical_generator_12.lexical_generator(url)
    # print(numerical_values)
    numerical_values = DMatrix(numerical_values)

    match xgb_classifier_12.predict(numerical_values):
        case 0:
            return "Benign"
        case 1:
            return "Malware"
        case 2:
            return "Phishing"
        case 3:
            return "Defacement"
        
def xgb_predict_maliciousness_33(url):

    numerical_values = lexical_generator_33.lexical_generator(url)
    # print(numerical_values)
    numerical_values = DMatrix(numerical_values)

    match xgb_classifier_33.predict(numerical_values):
        case 0:
            return "Benign"
        case 1:
            return "Malware"
        case 2:
            return "Phishing"
        case 3:
            return "Defacement"

url = "www.facebook.com/"
print("Current URL: "+url)

print("------------- Wrapper-Based (12 Features) -------------")
for i in range(15):
    start = time.perf_counter()
    prediction = xgb_predict_maliciousness_12(url)
    end = time.perf_counter()
    print("Trial "+str(i))
    print(prediction)
    print(end-start)

print("------------- Wrapper-Based (33 Features) -------------")
for i in range(15):
    start = time.perf_counter()
    prediction = xgb_predict_maliciousness_33(url)
    end = time.perf_counter()
    print("Trial "+str(i))
    print(prediction)
    print(end-start)

Current URL: www.facebook.com/
------------- Wrapper-Based (12 Features) -------------
Trial 0
Benign
0.01231230003759265
Trial 1
Benign
0.0145448999828659
Trial 2
Benign
0.012852000014390796
Trial 3
Benign
0.015412199951242656
Trial 4
Benign
0.013804399990476668
Trial 5
Benign
0.013748899975325912
Trial 6
Benign
0.012871399987488985
Trial 7
Benign
0.013846200017724186
Trial 8
Benign
0.015101300028618425
Trial 9
Benign
0.015125099977012724
Trial 10
Benign
0.013883600011467934
Trial 11
Benign
0.013928800006397069
Trial 12
Benign
0.013406700047198683
Trial 13
Benign
0.013778600026853383
Trial 14
Benign
0.012641800043638796
------------- Wrapper-Based (33 Features) -------------
Trial 0
Benign
0.019543200032785535
Trial 1
Benign
0.01978229999076575
Trial 2
Benign
0.019805500051006675
Trial 3
Benign
0.019335700024385005
Trial 4
Benign
0.01775130000896752
Trial 5
Benign
0.018495200027246028
Trial 6
Benign
0.018937400018330663
Trial 7
Benign
0.019897000049240887
Trial 8
Benign
0.020229200017